In [2]:
# Notwendige Imports
import argparse, csv, logging, time, urllib.parse
import orjson, pandas as pd, requests
from __future__ import annotations
from pathlib import Path
from SPARQLWrapper import SPARQLWrapper, JSON

In [11]:
# Basis-URL für ORCID API v3.0
ORCID_BASE = "https://pub.orcid.org/v3.0"

# HTTP-Header zur Angabe des gewünschten Antwortformats (JSON)
HEADERS    = {"Accept": "application/json"}

# Dateipfade
BASE_DIR = Path.cwd()
DATA_FILE  = BASE_DIR / "import" / "NFDI4Microbiota_staff_input.xlsx"
OUT_FILE   = BASE_DIR / "import" / "input_with_orcid.csv"

# Sekunden zwischen API‑Calls
RATE_SLEEP = 0.25

In [12]:
"""
Führt einen HTTP-GET-Request auf die angegebene URL aus und gibt die Antwort als JSON zurück.
Es werden bis zu drei Versuche unternommen, falls ein Fehler auftritt (z. B. Verbindungsfehler, Timeout, HTTP-Fehler).
Die Antwort wird mithilfe von `orjson` dekodiert.
"""

def _get_json(url: str) -> dict:
    # Wiederhole bis zu drei Versuche bei Fehlern
    for attempt in range(3):
        try:
            # Führe HTTP-GET mit gesetztem Header und Timeout durch
            r = requests.get(url, headers=HEADERS, timeout=20)

            # Löst bei HTTP-Fehlercodes (4xx, 5xx) eine Exception aus
            r.raise_for_status()

            # Dekodiere und gib JSON-Antwort zurück
            return orjson.loads(r.content)

        # Bei beliebigen Netzwerk-/HTTP-Problemen: loggen und erneut versuchen
        except requests.exceptions.RequestException as exc:
            logging.warning("ORCID call failed (%s/3): %s", attempt + 1, exc)
            time.sleep(1)

    # Nach drei Fehlversuchen: gib leeres Dict zurück
    return {}

In [13]:
"""
Funktion, die nach einer ORCID-ID basierend auf Namen und optionaler Organisation sucht.

Strategie:
    1. Suche mit Vor- und Nachname.
    2. Falls eine Organisation (`org`) angegeben ist, wird ein zweiter Versuch mit Affiliation-Filter durchgeführt.

Die Funktion gibt die erste gefundene ORCID-ID zurück oder `None`, falls keine Treffer erzielt wurden.
"""

def orcid_search(given: str, family: str, org: str | None = None, debug: bool = False) -> str | None:
    # Ohne Nachname keine Abfrage möglich – ORCID verlangt Vor- und Nachnamen
    if not family:
        return None

    # Baue Basis-Query: Vorname + Nachname
    base_q = f'given-names:"{given}"+AND+family-name:"{family}"'
    queries = [base_q]

    # Wenn Organisation angegeben, erweitere die Suchanfrage
    if org:
        queries.append(f'{base_q} AND affiliation-org-name:"{org}"')

    # Versuche jede definierte Query nacheinander
    for q in queries:
        # Ersetze Leerzeichen durch '+' für ORCID-kompatible Syntax
        q_plus = q.replace(' ', '+')

        # URI-Encoding: '+' darf nicht maskiert werden, ':' und '"' ebenfalls nicht
        encoded_q = urllib.parse.quote(q_plus, safe=':"+')

        # Zusammensetzen der vollständigen API-URL mit Suchbegriff
        url = f"{ORCID_BASE}/expanded-search/?q={encoded_q}&rows=5"

        # Optional: zeige die verwendete Query im Klartext
        if debug:
            print("🚀 Query:", urllib.parse.unquote(url))

        # Sende API-Anfrage und lade JSON-Daten
        data = _get_json(url)

        # Optional: Rohstruktur der Antwort inspizieren
        if debug:
            print("  ↳ raw keys:", list(data.keys()))

        # Ergebnis extrahieren – ORCID verwendet manchmal unterschiedliche Keys
        hits = data.get("result") or data.get("expanded-result")

        # Optional: Anzahl gefundener Treffer anzeigen
        if debug:
            print("  ↳ result count:", len(hits or []))

        # Wartezeit zur Entlastung der API
        time.sleep(RATE_SLEEP)

        # Gib ORCID-ID des ersten Treffers zurück
        if hits:
            return hits[0]["orcid-id"]

    # Kein Treffer → None
    return None

In [14]:
"""
Führt eine SPARQL-Abfrage auf Wikidata durch, um die ORCID-ID einer Person anhand ihres Namens zu ermitteln.
Die Abfrage sucht case-insensitiv nach einem exakten Label-Match (rdfs:label) in Wikidata und gibt die zugehörige ORCID-ID (P496) zurück.
"""

def scholia_orcid(full_name: str) -> str | None:
    # Initialisiere SPARQL-Abfrage über Wikidata-Endpunkt
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

    # Antwortformat auf JSON setzen
    sparql.setReturnFormat(JSON)

    # SPARQL-Abfrage definieren: suche Label unabhängig von Groß-/Kleinschreibung
    sparql.setQuery(f'''
        SELECT ?orcid WHERE {{
          ?person wdt:P496 ?orcid ;
                  rdfs:label ?lab .
          FILTER(LCASE(STR(?lab)) = "{full_name.lower()}")
        }} LIMIT 1''')

    try:
        # Anfrage ausführen und Ergebnis extrahieren
        res = sparql.query().convert()["results"]["bindings"]

        # Gib die ORCID-ID zurück, falls Treffer vorhanden
        return res[0]["orcid"]["value"] if res else None

    except Exception as exc:
        # Fehler bei SPARQL-Ausführung → Debug-Log + None
        logging.debug("SPARQL error for %s: %s", full_name, exc)
        return None

In [15]:
"""
Führt eine ORCID-Anreicherung für eine Excel-Liste von Personen durch.

Für jede Person (Name + Institution) wird versucht, eine passende ORCID-ID über die ORCID-API oder Wikidata zu finden.
Das Ergebnis wird in eine CSV-Datei exportiert.
"""

# limit (int | None, optional): Anzahl der zu verarbeitenden Personen (zum Testen o. Debuggen).
DEFAULT_LIMIT = 5

def run(limit: int | None = DEFAULT_LIMIT):
    logging.info("📥  Lade Staff‑Liste …")

    # Einlesen der Eingabedatei (Excel)
    df = pd.read_excel(DATA_FILE)

    # Ergebniszeilen für die CSV-Ausgabe
    rows = []

    for idx, r in enumerate(df.itertuples(index=False), start=1):

        # Verarbeitung nach 'limit' Einträgen abbrechen
        if limit and idx > limit:
            break

        # Vor- und Nachnamen trennen
        parts = str(r.Name).strip().split()
        given  = parts[0]
        family = " ".join(parts[1:])

        logging.info("▶ [%s/%s] %s", idx, len(df), r.Name)

        # ORCID-Suche via offizielle API, optional mit Institution
        orcid_id = orcid_search(given, family, r.Institution)

        # Fallback: ORCID-Suche über Wikidata (SPARQL)
        if not orcid_id:
            orcid_id = scholia_orcid(r.Name)

        # Eintrag in Ergebnisliste aufnehmen
        rows.append({
            "Institution":  r.Institution,
            "Name":         r.Name,
            "ORCID":        orcid_id or "",
            "ORCID-Link":   f"https://orcid.org/{orcid_id}" if orcid_id else ""
        })

    logging.info("💾  Schreibe %s", OUT_FILE)

    # Ergebnis in CSV-Datei schreiben
    pd.DataFrame(rows).to_csv(OUT_FILE, index=False, quoting=csv.QUOTE_ALL)

    logging.info("✅  Fertig – %s Zeilen", len(rows))

In [16]:
if __name__ == "__main__":

    # Konfiguriere das Logging-Format und -Level
    logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

    # Kommandozeilenargumente parsen
    parser = argparse.ArgumentParser()
    parser.add_argument("--limit", type=int, help="Nur N Zeilen verarbeiten")

    # Argumente aus sys.argv lesen (überspringt unbekannte Argumente)
    args, _ = parser.parse_known_args(sys.argv[1:])

    # Starte den Hauptprozess mit dem angegebenen Limit
    run(args.limit)

INFO: 📥  Lade Staff‑Liste …
INFO: ▶ [1/70] Alexander Sczyrba
INFO: ▶ [2/70] Jens Stoye
INFO: ▶ [3/70] Michael Beckstette
INFO: ▶ [4/70] Nils Kleinbölting
INFO: ▶ [5/70] Liren Huang
INFO: ▶ [6/70] Sebastian Jünemann
INFO: ▶ [7/70] Kassian Kobert
INFO: ▶ [8/70] Anandhi Iyappan
INFO: ▶ [9/70] Peer Bork
INFO: ▶ [10/70] Sina Barysch
INFO: ▶ [11/70] Sarah Schulz
INFO: ▶ [12/70] Daniel Podlesny
INFO: ▶ [13/70] Mahdi Robbani
INFO: ▶ [14/70] Noriko Cassman
INFO: ▶ [15/70] Shahram Saghaei
INFO: ▶ [16/70] Sandra Triebel
INFO: ▶ [17/70] Kilian Ossetek
INFO: ▶ [18/70] Manja Marz
INFO: ▶ [19/70] Winfried Göttsch
INFO: ▶ [20/70] Anderson Santos
INFO: ▶ [21/70] Jonas Kasmanas
INFO: ▶ [22/70] Stefanía Magnúsdóttir
INFO: ▶ [23/70] Majid Soheilie
INFO: ▶ [24/70] Sanchita Kamath
INFO: ▶ [25/70] Nathan Ernster
INFO: ▶ [26/70] Ulisses Nunes da Rocha
INFO: ▶ [27/70] Martin Bole
INFO: ▶ [28/70] Adrian Fritz
INFO: ▶ [29/70] Alice McHardy
INFO: ▶ [30/70] Mattea Müller
INFO: ▶ [31/70] Carmen Paulmann
INFO: ▶ [32

In [17]:
# Testaufruf
# orcid1 = orcid_search("Konrad", "Förstner", debug=True)
# print("ORCID1:", orcid1)